# POST Create, Read, Update and Delete (CRUD)

- In this chapter, we will use class based view to implement Post Create, Detail, Update and Delete functions

## Introduction to Class-Based Views (CBVs) vs. Function-Based Views (FBVs)

### Class-Based Views (CBVs):

Advantages:

- **Reusable**: You can inherit and reuse views.
- **Organized**: Code is more structured, separating concerns into methods.
- **Built-in functionality**: CBVs provide built-in methods for common tasks.
  Disadvantages:
- **Complexity**: More complex to understand initially.
- **Less control**: Overriding built-in behavior can be complex.

### Function-Based Views (FBVs):

Advantages:

- **Simplicity**: Easier to understand and use.
- **Flexibility**: More straightforward for simple views.
  Disadvantages:
- **Less reusable**: Harder to reuse and extend.
- **Less organized**: Can become cluttered with increasing complexity.


## Step 1: Create Post Create View

- To create a new post with fields for the title and content.
- The view will ensure that the post is **associated with the logged-in `user`**.

  ```python
  from django.views.generic import CreateView
  from django.contrib.auth.mixins import LoginRequiredMixin
  from .models import Post

  class PostCreateView(LoginRequiredMixin, CreateView):
      model = Post
      fields = ['title', 'content']

      def form_valid(self, form):
          form.instance.author = self.request.user
          return super().form_valid(form)

  ```

### Notes:

- **Form Validation**: The `form_valid` method ensures the form is valid and associates the post with the logged-in user.
- **Template Naming Convention**: The template for this view should be named `post_form.html` by default.


## Step 2: Create Post Update View

- To update an existing post.
- This view uses the same template as the **create view** and includes **privilege checks to ensure only the post author can update the post**.

  ```python
  from django.views.generic import UpdateView
  from django.contrib.auth.mixins import UserPassesTestMixin

  ...

  class PostUpdateView(LoginRequiredMixin, UserPassesTestMixin, UpdateView):
      model = Post
      fields = ['title', 'content']

      def form_valid(self, form):
          form.instance.author = self.request.user
          return super().form_valid(form)

      def test_func(self):
          post = self.get_object()
          if post.author == self.request.user:
              return True
          return False

  ```

### Notes:

- **Privilege Check**: The `test_func` method ensures that only the post author can update the post.


## Step 3: Set Template for Post Create and Update Views

* `users/templates/post_form.html`

    ```html
    {% extends "blog/main.html" %}
    {% load crispy_forms_tags %}

    {% block main-content %}

    <div class="container">
        <h2>Post a Blog</h2>
        {% for message in messages %}
        <div class="alert alert-{{ message.tags }}">
            {{ message }}
        </div>
        {% endfor %}
        <form method="POST">
            {% csrf_token %}
            {{ form|crispy }}
            <div class="d-flex justify-content-end mt-2">
                <button type="submit" class="btn btn-primary me-2">Post</button>
                <a href="{% url 'blog-home' %}" class="btn btn-danger">Cancel</a>
            </div>
        </form>
    </div>

    {% endblock main-content %}
    ```

## Step4: Set Post Delete View

- To delete an existing post.
- The view ensures only the post author can delete the `post`.

  ```python
  from django.views.generic import DeleteView
  from django.urls import reverse_lazy

  class PostDeleteView(LoginRequiredMixin, UserPassesTestMixin, DeleteView):
      model = Post
      success_url = reverse_lazy('blog-home')

      def test_func(self):
          post = self.get_object()
          if post.author == self.request.user:
              return True
          return False

  ```

### Notes:

- **Success URL**: The `success_url` attribute defines the URL to redirect to after successful deletion.


## Step 5: Construct Post Confirm Delete Template

* `users/templates/post_confirm_delete.html`

    ```html
    {% extends "blog/main.html" %}
    {% load crispy_forms_tags %}

    {% block main-content %}

    <div class="container">
        <h2>Delete Blog</h2>
        {% for message in messages %}
        <div class="alert alert-{{ message.tags }}">
            {{ message }}
        </div>
        {% endfor %}
        <h2>Are you sure you want to delete the post "{{ object.title }}?"</h2>
        <form method="POST">
            {% csrf_token %}
            {{ form|crispy }}
            <div class="d-flex justify-content-end mt-2">
                <button type="submit" class="btn btn-danger me-2">Delete</button>
                <a href="{% url 'post-detail' object.pk %}" class="btn btn-secondary">Cancel</a>
            </div>
        </form>
    </div>

    {% endblock main-content %}

    ```

## Step 6: Set Post Detail View

- To display the details of a specific post.

    ```python
    from django.views.generic import DetailView
    from .models import Post

    class PostDetailView(DetailView):
        model = Post
    ```


## Step 7: Construct Post Detail Template

* `users/templates/users/post_detail.html`

    ```html
    {% extends 'blog/main.html' %} {% block main-content%}

    <article class="media content-section">
        <img class="rounded-circle article-img" src="{{ object.author.userprofile.image.url }}" />
        <div class="media-body">
            <div class="article-metadata">
                <a class="mr-2" href="#">{{ object.author }}</a>
                <small class="text-muted">{{ object.date_posted|date:"F d, Y" }}</small>

                {% if object.author == user %}
                <a class="btn btn-secondary btn-sm mt-1 mb-1" href="{% url 'post-update' object.pk %}">Update</a>
                <a class="btn btn-danger btn-sm mt-1 mb-1" href="{% url 'post-delete' object.pk %}">Delete</a>
                {% endif %}

            </div>
            <h2 class="article-title">{{ object.title }}</h2>
            <p class="article-content">{{ object.content }}</p>
        </div>
    </article>

    {% endblock main-content %}

    ```

### Notes:

* **Privilege Check**: Ensure that only the author can see the update and delete options.

## Step 8: Construct URL Patterns

- Define the URL patterns for the post CRUD operations.

  ```python
  from django.urls import path
  from .views import (
      PostDetailView,
      PostCreateView,
      PostUpdateView,
      PostDeleteView
  )
  from . import views

  urlpatterns = [
      ...
      path("post/<int:pk>", PostDetailView.as_view(), name='post-detail'),
      path("post/<int:pk>/update", PostUpdateView.as_view(), name='post-update'),
      path("post/<int:pk>/delete", PostDeleteView.as_view(), name='post-delete'),
      path("post/new", PostCreateView.as_view(), name='post-create'),
      ...
  ]

  ```

* Write `get_absolute_url` in Post Model

  - The `get_absolute_url` method provides a way to generate a URL for the detail view of the object.
  - This is particularly useful in `templates` and `views` where you **need to link to the detailed view of a model instance**.
  - The `get_absolute_url` method uses the `reverse` function to generate the URL for the detailed view of the post.
  - The `reverse` function takes the name of the URL pattern (`post-detail`) and the `primary key` of the post as arguments.

  ```python
  from django.db import models
  from django.contrib.auth.models import User
  from django.utils import timezone
  from django.urls import reverse

  class Post(models.Model):
      title = models.CharField(max_length=80)
      author = models.ForeignKey(User, on_delete=models.CASCADE)
      content = models.TextField()
      date_posted = models.DateTimeField(default=timezone.now)

      def __str__(self):
          return self.title

      def get_absolute_url(self):
          return reverse('post-detail', kwargs={'pk': self.pk})
  ```


## Additional Resource

### Built-in generic views

<table>
  <tr>
    <th>View</th>
    <th>Description</th>
    <th>Parameters</th>
    <th>Example</th>
  </tr>
  <tr>
    <td>TemplateView</td>
    <td>Renders a template.</td>
    <td>
      <ul>
        <li><b>template_name:</b> The name of the template to use.</li>
        <li><b>extra_context:</b> A dictionary of context data to add to the template context.</li>
      </ul>
    </td>
    <td>
      <pre>
python
from django.views.generic import TemplateView
class HomePageView(TemplateView):
    template_name = 'home.html'
      </pre>
    </td>
  </tr>
  <tr>
    <td>ListView</td>
    <td>Displays a list of objects.</td>
    <td>
      <ul>
        <li><b>model:</b> The model to query.</li>
        <li><b>template_name:</b> The name of the template to use.</li>
        <li><b>context_object_name:</b> The name of the context variable to use.</li>
        <li><b>paginate_by:</b> Number of objects per page.</li>
      </ul>
    </td>
    <td>
      <pre>
python
from django.views.generic import ListView
from .models import Post
class PostListView(ListView):
    model = Post
    template_name = 'posts/post_list.html'
    context_object_name = 'posts'
      </pre>
    </td>
  </tr>
  <tr>
    <td>DetailView</td>
    <td>Displays a detail page for a single object.</td>
    <td>
      <ul>
        <li><b>model:</b> The model to query.</li>
        <li><b>template_name:</b> The name of the template to use.</li>
        <li><b>context_object_name:</b> The name of the context variable to use.</li>
      </ul>
    </td>
    <td>
      <pre>
python
from django.views.generic import DetailView
from .models import Post
class PostDetailView(DetailView):
    model = Post
    template_name = 'posts/post_detail.html'
    context_object_name = 'post'
      </pre>
    </td>
  </tr>
  <tr>
    <td>CreateView</td>
    <td>Displays a form for creating a new object and saves the object.</td>
    <td>
      <ul>
        <li><b>model:</b> The model to query.</li>
        <li><b>form_class:</b> The form class to use.</li>
        <li><b>template_name:</b> The name of the template to use.</li>
        <li><b>success_url:</b> URL to redirect to after success.</li>
      </ul>
    </td>
    <td>
      <pre>
python
from django.views.generic import CreateView
from .models import Post
from .forms import PostForm
class PostCreateView(CreateView):
    model = Post
    form_class = PostForm
    template_name = 'posts/post_form.html'
    success_url = '/posts/'
      </pre>
    </td>
  </tr>
  <tr>
    <td>UpdateView</td>
    <td>Displays a form for updating an existing object and saves the changes.</td>
    <td>
      <ul>
        <li><b>model:</b> The model to query.</li>
        <li><b>form_class:</b> The form class to use.</li>
        <li><b>template_name:</b> The name of the template to use.</li>
        <li><b>success_url:</b> URL to redirect to after success.</li>
      </ul>
    </td>
    <td>
      <pre>
python
from django.views.generic import UpdateView
from .models import Post
from .forms import PostForm
class PostUpdateView(UpdateView):
    model = Post
    form_class = PostForm
    template_name = 'posts/post_form.html'
    success_url = '/posts/'
      </pre>
    </td>
  </tr>
  <tr>
    <td>DeleteView</td>
    <td>Displays a confirmation page and deletes an existing object.</td>
    <td>
      <ul>
        <li><b>model:</b> The model to query.</li>
        <li><b>template_name:</b> The name of the template to use.</li>
        <li><b>success_url:</b> URL to redirect to after success.</li>
      </ul>
    </td>
    <td>
      <pre>
python
from django.views.generic import DeleteView
from .models import Post
class PostDeleteView(DeleteView):
    model = Post
    template_name = 'posts/post_confirm_delete.html'
    success_url = '/posts/'
      </pre>
    </td>
  </tr>
</table>
